In [2]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [3]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [3]:
design.delete_all_components()

In [5]:
design.get_chip_z('Q_chip')

'5um'

In [4]:
design.chips['C_chip'].size['size_x']='8mm'
design.chips['C_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['size_x']='8mm'
design.chips['Q_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['center_z']='5um'

# Resonator length

In [6]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [7]:
resonator_length_2=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,2)
    a=a.strip('mm')
    a=float(a)-0.35
    resonator_length_2.append(a)
print(resonator_length_2)


[8.103254683870285, 7.874788339841421, 7.658346540209131]


In [12]:
resonator_length_4=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,4)
    a=a.strip('mm')
    a=float(a)
    resonator_length_4.append(a)
print(resonator_length_4)


[4.226627341935142, 4.1123941699207105, 4.004173270104565]


# Flipchip

In [8]:
cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))
cp1 = dict(down = dict(loc_W=-1, loc_H=-1, pad_width='120um'))

q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
                                            pos_y='-2000um',
                                            chip='Q_chip',
                                            connection_pads=cp0))]
q1=[TransmonPocket6(design, 'Q1', options=dict(pos_x='2000um',
                                            pos_y='-2000um',
                                            chip='Q_chip',
                                            connection_pads=cp1))]
qubits=q0+q1

q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))
q1x=float(qubits[1].options.pos_x.strip('um'))
q1y=float(qubits[1].options.pos_y.strip('um'))

c0 = LeftCoupler(design, "c0", options=dict(pos_x=f"{q0x-100}um",
                                              pos_y=f"{q0y+100}um",
                                              pad_width="150um",
                                              pad_height="80um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))
c1 = RightCoupler(design, "c1", options=dict(pos_x=f"{q1x+100}um",
                                              pos_y=f"{q1y+100}um",
                                              pad_width="150um",
                                              pad_height="80um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))  

design.rebuild()
gui.rebuild()
gui.autoscale()

In [9]:
#planar랑 똑같은 디자인
cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))
cp1 = dict(down = dict(loc_W=-1, loc_H=-1, pad_width='120um'))

q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
                                            pos_y='-2000um',
                                            chip='Q_chip',
                                            connection_pads=cp0))]
q1=[TransmonPocket6(design, 'Q1', options=dict(pos_x='2000um',
                                            pos_y='-2000um',
                                            chip='Q_chip',
                                            connection_pads=cp1))]
qubits=q0+q1

q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))
q1x=float(qubits[1].options.pos_x.strip('um'))
q1y=float(qubits[1].options.pos_y.strip('um'))

c0 = LeftCoupler(design, "c0", options=dict(pos_x=f"{q0x-150}um",
                                              pos_y=f"{q0y+140}um",
                                              pad_width="150um",
                                              pad_height="30um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))
c1 = RightCoupler(design, "c1", options=dict(pos_x=f"{q1x+150}um",
                                              pos_y=f"{q1y+140}um",
                                              pad_width="150um",
                                              pad_height="30um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))  

design.rebuild()
gui.rebuild()
gui.autoscale()

# Launchpad

In [10]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-3000um',
                                         pos_y = '2000um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '3000um',
                                         pos_y = '2000um',
                                         orientation='180',
                                         lead_length = '30um'))

# Feedline

In [11]:
clt0 = CoupledLineTee(design, "clt0", 
                        options=dict(pos_x='-2000um',
                                    pos_y='2000um', 
                                    coupling_length='300um',
                                    chip="C_chip",
                                    down_length='300um',
                                    mirror=True,
                                    open_termination=True))
clt1 = CoupledLineTee(design, "clt1", 
                        options=dict(pos_x="2000um",
                                    pos_y='2000um',
                                    coupling_length='300um', 
                                    chip="C_chip",
                                    down_length='300um',
                                    mirror=False,
                                    open_termination=True))


In [12]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_start'))))
fl1=RouteStraight(design, 'fl1',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='clt1',
                                                            pin='prime_start'))))
fl2=RouteStraight(design, 'fl2',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt1',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))

# Readout Resonator

In [13]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

In [14]:
rr0=connect('rr0', 'clt0', 'second_end', 'c0', 'in',
            resonator_length_2[1],jogs,jogs, 
            '400um','0um', '100um', '100um')
rr1=connect('rr1', 'clt1', 'second_end', 'c1', 'in',
            resonator_length_2[0],jogs,jogs, 
            '400um','0um', '100um', '100um')
bus=connect('bus', 'Q0', 'down', 'Q1', 'down',
            resonator_length_2[2],jogs,jogs, 
            '300um','-800um', '100um', '100um')            

# LOM Analysis

In [15]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [16]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.18',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.18',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [23]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down


In [18]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('Q0', 'down'), ('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 10:53PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp9iepvfjy.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 4780000000, Maxwell, 1, False
INFO 10:53PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpr6hqhp8d.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4780000000, Maxwell, 1, False
INFO 10:53PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpqpn_hibd.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4780000000, Maxwell, 2, False
INFO 10:53PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpygu9vy6o.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4780000000, Maxwell, 3, False
INFO 10:53PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp8ewo2jzr.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4780000000, Maxwell, 4, False
INFO 10:53

,down_connector_pad_Q0,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
down_connector_pad_Q0,58.191000,-47.111,-9.77650,-0.91115,-0.011243
ground,-47.111000,474.680,-143.78000,-147.93000,-69.254000
pad_bot_Q0,-9.776500,-143.780,181.20000,-26.56200,-0.300380
pad_top_Q0,-0.911150,-147.930,-26.56200,178.94000,-2.864600
quad_coupler_c0,-0.011243,-69.254,-0.30038,-2.86460,72.586000


In [19]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 7.79
c1.setup.freq_bus = [9.06]
c1.run_lom()
c1.lumped_oscillator_all

[2, 3] [4 0]
Predicted Values

Transmon Properties
f_Q 4.491471 [GHz]
EC 184.568236 [MHz]
EJ 14.854145 [GHz]
alpha -204.704561 [MHz]
dispersion 0.063400 [KHz]
Lq 10.995564 [nH]
Cq 104.948872 [fF]
T1 9980.572685 [us]

**Coupling Properties**

tCqbus1 -1.293251 [fF]
gbus1_in_MHz -15.325097 [MHz]
χ_bus1 -0.008969 [MHz]
1/T1bus1 8.995801 [Hz]
T1bus1 17692.136799 [us]

tCqbus2 4.395055 [fF]
gbus2_in_MHz 60.366155 [MHz]
χ_bus2 -0.076664 [MHz]
1/T1bus2 6.950673 [Hz]
T1bus2 22897.774651 [us]
Bus-Bus Couplings
gbus1_2 0.535224 [MHz]


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.785559,315.006642,14.854145,-363.708763,20.399476,"[-77.27904326975295, 35.666365787491294]","[-0.9393967038668886, -0.0819795116208736]",0.939397,77.279043
2,4.688951,201.994337,14.854145,-225.315547,0.188611,"[-3.399918230634072, 53.061463039263096]","[-0.0005390519612973164, -0.06997454323891793]",0.000539,3.399918
3,4.555942,190.162117,14.854145,-211.299384,0.09146,"[-9.991297462572216, 59.43866791025907]","[-0.004067672800533768, -0.07847895920529005]",0.004068,9.991297
4,4.50999,186.165602,14.854145,-206.585679,0.070513,"[-10.019036046333751, 58.167978931204956]","[-0.003905469471436769, -0.07230266056983248]",0.003905,10.019036
5,4.51181,186.323013,14.854145,-206.771142,0.071251,"[-12.596885500784968, 59.232375504819544]","[-0.006185027980067806, -0.07508809522970526]",0.006185,12.596886
6,4.51284,186.41218,14.854145,-206.876206,0.071672,"[-14.019730150415121, 59.53769930631746]","[-0.007669097105811101, -0.07593015261481789]",0.007669,14.019730
7,4.508314,186.020737,14.854145,-206.415013,0.06984,"[-14.367782018451786, 60.892921658185415]","[-0.0080180719773452, -0.07912365653514689]",0.008018,14.367782
8,4.495078,184.87874,14.854145,-205.070092,0.064731,"[-15.008417526755165, 60.71480831161509]","[-0.008633583364216215, -0.07778857259273392]",0.008634,15.008418
9,4.491471,184.568236,14.854145,-204.704561,0.0634,"[-15.325097171718436, 60.366154997329936]","[-0.008969263931573, -0.07666429435407236]",0.008969,15.325097


In [21]:
#c1.sim.run(components=['Q1','c1'], open_terminations=[('Q1', 'down'), ('c1', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 11:03PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp8tsf1czv.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 4810000000, Maxwell, 1, False
INFO 11:03PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmphm2ru9n1.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4810000000, Maxwell, 1, False
INFO 11:03PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp1x3jg5j6.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4810000000, Maxwell, 2, False
INFO 11:03PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpyvcdrr5e.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4810000000, Maxwell, 3, False
INFO 11:03PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmppi3_n9ly.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4810000000, Maxwell, 4, False
INFO 11:03

,down_connector_pad_Q1,ground,pad_bot_Q1,pad_top_Q1,quad_coupler_c1
down_connector_pad_Q1,57.174000,-46.337,-9.58890,-0.91982,-0.010547
ground,-46.337000,471.790,-142.83000,-146.42000,-68.485000
pad_bot_Q1,-9.588900,-142.830,179.61000,-26.50100,-0.293530
pad_top_Q1,-0.919820,-146.420,-26.50100,177.01000,-2.716100
quad_coupler_c1,-0.010547,-68.485,-0.29353,-2.71610,71.613000


In [22]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 7.60
c1.setup.freq_bus = [9.06]
c1.run_lom()
c1.lumped_oscillator_all

[2, 3] [4 0]
Predicted Values

Transmon Properties
f_Q 4.507118 [GHz]
EC 185.917434 [MHz]
EJ 14.854145 [GHz]
alpha -206.293318 [MHz]
dispersion 0.069364 [KHz]
Lq 10.995564 [nH]
Cq 104.187260 [fF]
T1 10138.729062 [us]

**Coupling Properties**

tCqbus1 -1.224488 [fF]
gbus1_in_MHz -14.237163 [MHz]
χ_bus1 -0.008776 [MHz]
1/T1bus1 8.927096 [Hz]
T1bus1 17828.299021 [us]

tCqbus2 4.288438 [fF]
gbus2_in_MHz 59.237308 [MHz]
χ_bus2 -0.074804 [MHz]
1/T1bus2 6.770625 [Hz]
T1bus2 23506.684296 [us]
Bus-Bus Couplings
gbus1_2 0.492788 [MHz]


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.423096,274.532378,14.854145,-313.204209,5.366265,"[-67.03890464038305, 70.21534909679127]","[-0.5363505067277822, -0.23001998093274292]",0.536351,67.038905
2,4.981833,229.449161,14.854145,-258.184728,0.808717,"[-4.660970367272075, 56.627017121287736]","[-0.0015619627106529224, -0.10218571311246558]",0.001562,4.660970
3,4.705034,203.45175,14.854145,-227.048212,0.205283,"[-11.599071128462809, 39.15039414294918]","[-0.007170595774612459, -0.03861520079817567]",0.007171,11.599071
4,4.561968,190.689681,14.854145,-211.922394,0.094597,"[-10.829133009465176, 54.23539210399388]","[-0.005376127340547895, -0.0656731880888456]",0.005376,10.829133
5,4.526914,187.632085,14.854145,-208.314124,0.077651,"[-12.954378798667912, 53.250057581136154]","[-0.0074171600527688, -0.061464418022155444]",0.007417,12.954379
6,4.518199,186.876168,14.854145,-207.423001,0.073896,"[-13.488128125927302, 54.808962316805705]","[-0.007968364259852109, -0.06463913685483927]",0.007968,13.488128
7,4.507118,185.917434,14.854145,-206.293318,0.069364,"[-14.23716326143334, 59.23730761227235]","[-0.00877626137426658, -0.07480413873138407]",0.008776,14.237163


# EPR Analysis

In [15]:
from qiskit_metal.analyses.quantization import EPRanalysis

In [16]:
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [17]:
hfss.start()

INFO 05:22PM [connect_project]: Connecting to Ansys Desktop API...
INFO 05:22PM [load_ansys_project]: 	Opened Ansys App
INFO 05:22PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 05:22PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project21
INFO 05:22PM [connect_design]: No active design found (or error getting active design).
INFO 05:22PM [connect]: 	 Connected to project "Project21". No design detected


True

In [18]:
hfss.activate_ansys_design("Flipchip_5um", 'eigenmode')
hfss.render_design(['Q0','Q1','bus','c0','c1','lpL','lpR','clt0','clt1','rr0','rr1','fl0','fl1','fl2'],
                    [('clt0','prime_end'),('clt1','prime_start'),
                    ('bus','start'),('bus','end')],
                    box_plus_buffer=True)

05:22PM 30s WARNING [activate_ansys_design]: The design_name=Flipchip_5um was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 05:22PM [connect_design]: 	Opened active design
	Design:    Flipchip_5um [Solution type: Eigenmode]
WARNING 05:22PM [connect_setup]: 	No design setup detected.
WARNING 05:22PM [connect_setup]: 	Creating eigenmode default setup.
INFO 05:22PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [46]:
hfss.close()

Warning! 12 COM references still alive
Ansys will likely refuse to shut down


True